In [3]:
from datetime import date
from dotenv import load_dotenv
from google.cloud import bigquery
from google.oauth2 import service_account
import os
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
load_dotenv()

True

In [190]:
options  = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options)

In [191]:
url = 'https://sede.madrid.es/portal/site/tramites/menuitem.62876cb64654a55e2dbd7003a8a409a0/?vgnextoid=e3b0234be7924710VgnVCM2000001f4a900aRCRD&vgnextchannel=b59637c190180210VgnVCM100000c90da8c0RCRD&vgnextfmt=pd'
driver.get(url)

In [192]:
rechazar = driver.find_element(By.XPATH, '//*[@id="iam-cookie-control-modal-action-secondary"]')
rechazar.click()

In [193]:
en_linea = driver.find_element(By.XPATH, '//*[@id="verTodas"]/div/div[1]/p/a')
en_linea.click()

In [194]:
driver.close()
driver.switch_to.window(driver.window_handles[0])

usuario_contrasena = driver.find_element(By.XPATH, '//*[@id="ContentFixedSection_uSecciones_divSections"]/section[1]/div[2]/div/div/div[2]/article[1]')
usuario_contrasena.click()

In [195]:
acceso = driver.find_element(By.XPATH, '//*[@id="acceso_pass"]')
acceso.click()

In [196]:
usuario = driver.find_element(By.XPATH, '//*[@id="correoelectronico"]')
usuario.send_keys(os.environ.get('user'))

contrasena = driver.find_element(By.XPATH, '//*[@id="contrasenia"]')
contrasena.send_keys(os.environ.get('pass'))
contrasena.submit()

In [197]:
actividades_dia_centro = driver.find_element(By.XPATH, '//*[@id="ContentFixedSection_uSecciones_divSections"]/section[3]/div[2]/div/div/div[2]/article[1]/div')
actividades_dia_centro.click()

In [198]:
disponibilidad = driver.find_element(By.XPATH, '//*[@id="ContentFixedSection_uAltaEventos_uCentrosSeleccionar_availability_filter_on"]')
disponibilidad.click()

In [199]:
time.sleep(1)
centros = driver.find_elements(By.XPATH, "//li[contains(@class, 'media pull-left')]")

In [200]:
columnas = ["Actividad", "Horario", "Rango de edad", "Duración", "Centro", "Dirección", ]
df = pd.DataFrame(columns=columnas)

for i in range(len(centros)):
    centro = centros[i]
    time.sleep(3)
    centro.click()
    time.sleep(3)

    nombre = driver.find_element(By.XPATH, '//*[@id="ContentFixedSection_uAltaEventos_divFacility"]/div/h2')
    direccion = driver.find_element(By.XPATH, '//*[@id="ContentFixedSection_uAltaEventos_divFacility"]/div/div[2]')
    elementos_h4 = driver.find_elements(By.XPATH, '//div[@class="collapse in"]//h4')

    actividades_con_info_adicional = []

    rango_edad_actual = None
    duracion_actual = None

    patron_rango_edad_flexible = re.compile(r"(?i)\b(?:De\s*)?(\d+)\s*a\s*(\d+)\s*años\b")
    patron_rango_edad_apartirde = re.compile(r"(?i)\bA\s*partir\s+de\s+(\d+)\s+años\b")
    patron_duracion = r"(\d+)\s*[´'`]"
    patron_hora = r"\d{1,2}:\d{2}"

    for elemento in elementos_h4:
        texto = elemento.text
        
        if re.match(patron_hora, texto):
            actividades_con_info_adicional[-1][1].append(texto)
        else:
            titulo_actual = texto
            rango_edad_actual = None
            duracion_actual = None
            elementos_p = elemento.find_elements(By.XPATH, './following-sibling::p')
            
            for elem_p in elementos_p:
                texto_p = elem_p.text
                resultado_rango_edad = re.search(patron_rango_edad_flexible, texto_p)
                
                if resultado_rango_edad:
                    rango_edad_actual = f"De {resultado_rango_edad.group(1)} a {resultado_rango_edad.group(2)} años"

                else:
                    resultado_rango_edad_apartirde = re.search(patron_rango_edad_apartirde, texto_p)
                    if resultado_rango_edad_apartirde:
                        rango_edad_actual = f"A partir de {resultado_rango_edad_apartirde.group(1)} años"
                        
                resultado_duracion = re.search(patron_duracion, texto_p)
                if resultado_duracion:
                    duracion_actual = f"{resultado_duracion.group(1)}'"

            actividades_con_info_adicional.append([titulo_actual, [], rango_edad_actual, duracion_actual])

    for actividad in actividades_con_info_adicional:
        titulo = actividad[0]
        horas = actividad[1]
        rango_edad = actividad[2]
        duracion = actividad[3]
        df.loc[len(df)] = [titulo, horas, rango_edad, duracion, nombre.text, direccion.text]

    back = driver.find_element(By.XPATH, '//*[@id="ContentFixedSection_uAltaEventos_divFacility"]/div/div[1]/span')
    back.click()
driver.quit()
df.insert(0, 'Fecha', date.today().strftime("%d-%m-%Y"))
df.Horario = df.Horario.apply(lambda x: ', '.join(x))
df.Duración = df.Duración.str.replace("'", " minutos")
df

,Fecha,Actividad,Horario,Rango de edad,Duración,Centro,Dirección
0,31/07/2023,Baile,11:15,De 15 a 64 años,60 minutos,Antonio Díaz Miguel,"Calle Joaquín Dicenta, 1, 28029, Madrid"
1,31/07/2023,Ciclo sala,11:15,De 15 a 64 años,45 minutos,Antonio Díaz Miguel,"Calle Joaquín Dicenta, 1, 28029, Madrid"
2,31/07/2023,Entrenamiento funcional,12:30,De 15 a 64 años,60 minutos,Antonio Díaz Miguel,"Calle Joaquín Dicenta, 1, 28029, Madrid"
3,31/07/2023,Entrenamiento funcional,20:15,De 15 a 64 años,30 minutos,Antonio Díaz Miguel,"Calle Joaquín Dicenta, 1, 28029, Madrid"
4,31/07/2023,Entrenamiento por intervalos,18:30,De 15 a 64 años,30 minutos,Antonio Díaz Miguel,"Calle Joaquín Dicenta, 1, 28029, Madrid"
5,31/07/2023,Estiramientos relajación,20:00,De 15 a 64 años,60 minutos,Antonio Díaz Miguel,"Calle Joaquín Dicenta, 1, 28029, Madrid"
6,31/07/2023,Fitness,"10:00, 21:00",De 15 a 64 años,60 minutos,Antonio Díaz Miguel,"Calle Joaquín Dicenta, 1, 28029, Madrid"
7,31/07/2023,Fuerza ABDOMEN,18:00,De 15 a 64 años,30 minutos,Antonio Díaz Miguel,"Calle Joaquín Dicenta, 1, 28029, Madrid"
8,31/07/2023,Fuerza GAP,"08:30, 17:30",De 15 a 64 años,60 minutos,Antonio Díaz Miguel,"Calle Joaquín Dicenta, 1, 28029, Madrid"
9,31/07/2023,Gimnasia suave,08:30,De 15 a 64 años,60 minutos,Antonio Díaz Miguel,"Calle Joaquín Dicenta, 1, 28029, Madrid"


In [201]:
liscen = ['Antonio Díaz Miguel', 'Barajas', 'Cerro Almodóvar', 'Concepción',
              'Cubo Gym Las Tablas', 'Cubo Gym Madrid Río', 'Cubo Gym Retiro', 'El Quijote',
              'Francisco Fernández Ochoa', 'Fuente del Berro', 'Hortaleza', 'Juan de Dios Román',
              'La Almudena', 'La Masó', 'Las Cruces', 'San Cristóbal']

listel = [913145437, 913294219, 913314047, 914039020, 915880432, 915880432, 915880432, 914599871,
          915883731, 915881369, 913821965, 914801231, 915882355, 917397555, 915888600, 917950911]

lisdis = ['Tetuán', 'Barajas', 'Villa de Vallecas', 'Ciudad Lineal',
            'Fuencarral-El Pardo', 'Centro', 'Retiro','Moncloa-Aravaca',
            'Carabanchel', 'Salamanca', 'Hortaleza', 'Villa de Vallecas',
            'Ciudad Lineal', 'Fuencarral - El Pardo', 'Latina', 'Villaverde']

contacto = pd.DataFrame({'Centro' : liscen,
                         'Distrito' : lisdis,
                         'Teléfono' : listel})

contacto

,Centro,Distrito,Teléfono
0,Antonio Díaz Miguel,Tetuán,913145437
1,Barajas,Barajas,913294219
2,Cerro Almodóvar,Villa de Vallecas,913314047
3,Concepción,Ciudad Lineal,914039020
4,Cubo Gym Las Tablas,Fuencarral-El Pardo,915880432
5,Cubo Gym Madrid Río,Centro,915880432
6,Cubo Gym Retiro,Retiro,915880432
7,El Quijote,Moncloa-Aravaca,914599871
8,Francisco Fernández Ochoa,Carabanchel,915883731
9,Fuente del Berro,Salamanca,915881369


In [202]:
df = pd.merge(df, contacto, on='Centro', how='left')
df

,Fecha,Actividad,Horario,Rango de edad,Duración,Centro,Dirección,Distrito,Teléfono
0,31/07/2023,Baile,11:15,De 15 a 64 años,60 minutos,Antonio Díaz Miguel,"Calle Joaquín Dicenta, 1, 28029, Madrid",Tetuán,913145437
1,31/07/2023,Ciclo sala,11:15,De 15 a 64 años,45 minutos,Antonio Díaz Miguel,"Calle Joaquín Dicenta, 1, 28029, Madrid",Tetuán,913145437
2,31/07/2023,Entrenamiento funcional,12:30,De 15 a 64 años,60 minutos,Antonio Díaz Miguel,"Calle Joaquín Dicenta, 1, 28029, Madrid",Tetuán,913145437
3,31/07/2023,Entrenamiento funcional,20:15,De 15 a 64 años,30 minutos,Antonio Díaz Miguel,"Calle Joaquín Dicenta, 1, 28029, Madrid",Tetuán,913145437
4,31/07/2023,Entrenamiento por intervalos,18:30,De 15 a 64 años,30 minutos,Antonio Díaz Miguel,"Calle Joaquín Dicenta, 1, 28029, Madrid",Tetuán,913145437
5,31/07/2023,Estiramientos relajación,20:00,De 15 a 64 años,60 minutos,Antonio Díaz Miguel,"Calle Joaquín Dicenta, 1, 28029, Madrid",Tetuán,913145437
6,31/07/2023,Fitness,"10:00, 21:00",De 15 a 64 años,60 minutos,Antonio Díaz Miguel,"Calle Joaquín Dicenta, 1, 28029, Madrid",Tetuán,913145437
7,31/07/2023,Fuerza ABDOMEN,18:00,De 15 a 64 años,30 minutos,Antonio Díaz Miguel,"Calle Joaquín Dicenta, 1, 28029, Madrid",Tetuán,913145437
8,31/07/2023,Fuerza GAP,"08:30, 17:30",De 15 a 64 años,60 minutos,Antonio Díaz Miguel,"Calle Joaquín Dicenta, 1, 28029, Madrid",Tetuán,913145437
9,31/07/2023,Gimnasia suave,08:30,De 15 a 64 años,60 minutos,Antonio Díaz Miguel,"Calle Joaquín Dicenta, 1, 28029, Madrid",Tetuán,913145437


In [ ]:
new_header = df.iloc[0]
df = df[1:]
df.columns = new_header

In [203]:
# Se puede guardar en csv para llevar un registro diario, yo no lo haría, sino que subiría a la base de datos directamente.

In [204]:
# df.to_csv(f'data/{date.today().strftime("%d-%m-%Y")}.csv', index=False, header=False)

# Pruebas

In [58]:
df = pd.read_csv('data/31-07-2023.csv')
df.head(5)

,Fecha,Actividad,Horario,Rango de edad,Duración,Centro,Dirección,Distrito,Teléfono
0,31-07-2023,Baile,11:15,De 15 a 64 años,60 minutos,Antonio Díaz Miguel,"Calle Joaquín Dicenta, 1, 28029, Madrid",Tetuán,913145437
1,31-07-2023,Ciclo sala,11:15,De 15 a 64 años,45 minutos,Antonio Díaz Miguel,"Calle Joaquín Dicenta, 1, 28029, Madrid",Tetuán,913145437
2,31-07-2023,Entrenamiento funcional,12:30,De 15 a 64 años,60 minutos,Antonio Díaz Miguel,"Calle Joaquín Dicenta, 1, 28029, Madrid",Tetuán,913145437
3,31-07-2023,Entrenamiento funcional,20:15,De 15 a 64 años,30 minutos,Antonio Díaz Miguel,"Calle Joaquín Dicenta, 1, 28029, Madrid",Tetuán,913145437
4,31-07-2023,Entrenamiento por intervalos,18:30,De 15 a 64 años,30 minutos,Antonio Díaz Miguel,"Calle Joaquín Dicenta, 1, 28029, Madrid",Tetuán,913145437


In [62]:
print(df[df['Dirección'].isnull()])

Empty DataFrame
Columns: [Fecha, Actividad, Horario, Rango de edad, Duración, Centro, Dirección, Distrito, Teléfono]
Index: []


In [63]:
key_path = '../../key.json'
credentials = service_account.Credentials.from_service_account_file(key_path)
client = bigquery.Client(credentials=credentials, project=credentials.project_id,)

df.to_gbq(destination_table= 'tripulacionesgrupo5.app_dataset.actividades',
          if_exists='append',
          table_schema = [{'name': 'Fecha', 'type': 'STRING'},
                          {'name': 'Actividad', 'type': 'STRING'},
                          {'name': 'Horario', 'type': 'STRING'},
                          {'name': 'Rango de edad', 'type': 'STRING'},
                          {'name': 'Duración', 'type': 'STRING'},
                          {'name': 'Centro', 'type': 'STRING'},
                          {'name': 'Dirección', 'type': 'STRING'},
                          {'name': 'Distrito', 'type': 'STRING'},
                          {'name': 'Teléfono', 'type': 'INTEGER'}],
          credentials= credentials)

GenericGBQException: Reason: 400 Required field Dirección cannot be null